In [1]:
%pip install tensorflow
%pip install matplotlib
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 8.3/8.3 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Loading the data

In [1]:
import glob

data_dir = 'data/generated'
data_files = glob.glob(data_dir + "/*.csv")

In [2]:
import pandas as pd

df_list = []

for file in data_files:
    data_frame = pd.read_csv(file, index_col=None, header=0)
    df_list.append(data_frame)

df = pd.concat(df_list, ignore_index=True)

print(df.shape)
print(df.columns)

(995, 2)
Index(['pregunta', 'clase'], dtype='object')


In [3]:
df_test = pd.read_csv('data/preguntas_test.csv', index_col=None, header=0)
print(df_test.shape)
print(df.columns)

(19, 2)
Index(['pregunta', 'clase'], dtype='object')


In [4]:
NUMBER_OF_CLASSES = df['clase'].nunique()
MIN_NUMBER_OF_QUESTIONS = df[df['clase'] == df.groupby('clase').size().idxmin()].shape[0]

In [5]:
df_raw = df.groupby('clase').apply(lambda x: x.sample(n=MIN_NUMBER_OF_QUESTIONS)).reset_index(drop=True)
print(df_raw.shape)

(900, 2)


In [6]:
classes_list = df['clase'].unique().tolist()
print(classes_list)

['Definicion', 'EntidadAbreviatura', 'EntdidadGrupo', 'EntidadLugar', 'EntidadLugarCapital', 'EntidadLugarCiudad', 'EntidadLugarPais', 'EntidadObjeto', 'EntidadPersona', 'Evento', 'NumericoCantidad', 'NumericoEdad', 'NumericoMedida', 'NumericoPeriodo', 'NumericoPorcentaje', 'TemporalAnyo', 'TemporalFecha', 'TemporalMes']


In [7]:
classes_df_1 = classes_df = pd.DataFrame(columns=classes_list).reset_index(drop=True)
classes_df_2 = classes_df = pd.DataFrame(columns=classes_list).reset_index(drop=True)

for index, row in df_raw.iterrows():
    classes_df_1.loc[len(classes_df_1)] = [1 if row['clase'] == c else 0 for c in classes_list]

for index, row in df_test.iterrows():
    classes_df_2.loc[len(classes_df_2)] = [1 if row['clase'] == c else 0 for c in classes_list]

In [8]:
classes_df_1.sample(3)

,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
381,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
166,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
557,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [9]:
classes_df_2.sample(3)

,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df_raw.sample(3)

,pregunta,clase
739,¿Cuál es el porcentaje de personas que utiliza...,NumericoPorcentaje
60,¿Quién es el actor/actriz más joven en ganar u...,EntdidadGrupo
769,¿En qué año se produjo la caída del Imperio Ro...,TemporalAnyo


In [11]:
df_test.sample(3)

,pregunta,clase
6,¿Qué empresa de automóviles produce el “Escara...,EntidadObjeto
17,¿En qué día cae el Día del Año Nuevo Chino?,TemporalFecha
8,¿Cómo murió Jimi Hendrix?,EntidadEvento


In [12]:
df_one_hot_encoded = pd.concat([df_raw, classes_df_1], axis=1).drop('clase', axis=1)
df_test_one_hot_encoded = pd.concat([df_test, classes_df_2], axis=1).drop('clase', axis=1)

In [13]:
df_one_hot_encoded.sample(3)

,pregunta,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
595,¿Hace cuánto tiempo aprendiste a conducir?,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
51,¿Cuál es el grupo que ha ganado más campeonato...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
546,¿Cuánto disminuye la eficiencia de un panel so...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [14]:
df_test_one_hot_encoded.sample(3)

,pregunta,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
15,¿Dónde fue fundada Greenpeace?,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,¿Qué es el Mossad?,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,¿Qué cargo detenta Ariel Sharon?,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


# Preprocessing the data

In [15]:
df_one_hot_encoded['pregunta'] = df_raw['pregunta'].str.lower()
df_test_one_hot_encoded['pregunta'] = df_test['pregunta'].str.lower()

In [17]:
df_one_hot_encoded = df_one_hot_encoded.sample(frac=1)
df_test_one_hot_encoded = df_test_one_hot_encoded.sample(frac=1)

# Model creation

In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, optimizers
import tensorflow as tf

In [19]:
SEED = 30072001
X_train, x_test, Y_train, y_test = train_test_split(df_one_hot_encoded['pregunta'], 
                                                    df_one_hot_encoded[df_raw.columns[1:]], 
                                                    test_size=0.1, 
                                                    random_state=SEED)

In [18]:
X_train = df_one_hot_encoded['pregunta']
Y_train = df_one_hot_encoded[df_one_hot_encoded.columns[1:]]

X_test = df_test_one_hot_encoded['pregunta']
Y_test = df_test_one_hot_encoded[df_test_one_hot_encoded.columns[1:]]

In [19]:
MAX_FEATURES = 150
SEQUENCE_LENGTH = 50

vectorize_layer = layers.TextVectorization(
    standardize=None, 
    max_tokens=MAX_FEATURES, 
    output_mode='int', 
    output_sequence_length=SEQUENCE_LENGTH)

vectorize_layer.adapt(X_train)

In [20]:
EMBEDDING_DIM = 8

model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(layers.Embedding(MAX_FEATURES + 1, EMBEDDING_DIM))
model.add(layers.Dense(100))
model.add(layers.Dense(75))
model.add(layers.Dropout(0.2))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(NUMBER_OF_CLASSES, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 50)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 50, 8)             1208      
                                                                 
 dense (Dense)               (None, 50, 100)           900       
                                                                 
 dense_1 (Dense)             (None, 50, 75)            7575      
                                                                 
 dropout (Dropout)           (None, 50, 75)            0         
                                                                 
 global_average_pooling1d (G  (None, 75)               0         
 lobalAveragePooling1D)                                 

In [21]:
model.compile(optimizer=optimizers.Adam(0.035), loss='binary_crossentropy', metrics=['accuracy' , tf.keras.metrics.AUC()])

In [22]:
history = model.fit(X_train, Y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/100
23/23 [==============================] - 3s 27ms/step - loss: 0.2756 - accuracy: 0.0458 - auc: 0.4976 - val_loss: 0.2179 - val_accuracy: 0.0611 - val_auc: 0.5288
Epoch 2/100
23/23 [==============================] - 0s 9ms/step - loss: 0.2062 - accuracy: 0.1278 - auc: 0.6601 - val_loss: 0.1862 - val_accuracy: 0.2500 - val_auc: 0.7778
Epoch 3/100
23/23 [==============================] - 0s 9ms/step - loss: 0.1754 - accuracy: 0.2958 - auc: 0.8189 - val_loss: 0.1591 - val_accuracy: 0.3944 - val_auc: 0.8759
Epoch 4/100
23/23 [==============================] - 0s 9ms/step - loss: 0.1586 - accuracy: 0.4194 - auc: 0.8694 - val_loss: 0.1481 - val_accuracy: 0.4500 - val_auc: 0.8943
Epoch 5/100
23/23 [==============================] - 0s 9ms/step - loss: 0.1369 - accuracy: 0.5681 - auc: 0.9146 - val_loss: 0.1278 - val_accuracy: 0.6222 - val_auc: 0.9269
Epoch 6/100
23/23 [==============================] - 0s 9ms/step - loss: 0.1167 - accuracy: 0.7069 - auc: 0.9365 - val_loss: 0.1083 - 

In [119]:
metrics = model.evaluate(X_test, Y_test)
print("{}: {}".format(model.metrics_names[1], metrics[1]))

1/1 [==============================] - 0s 40ms/step - loss: 2.0479 - accuracy: 0.2632 - auc_15: 0.6526
accuracy: 0.2631579041481018


In [26]:
def interpret(r):
    max_prob = float('-inf')
    index = -1
    for i, prob in enumerate(r[0]):
        if (prob > max_prob):
            max_prob = prob
            index = i
    
    print("Clasificación: " + classes_list[index] + " con un " + max_prob.astype('str') + " de probabilidad.")



In [27]:
for q in X_test:
    print(q)
    interpret(model.predict([q]))
    print("\n")

¿qué año le fue concedido el premio nobel a thomas mann?
1/1 [==============================] - 0s 143ms/step
Clasificación: TemporalAnyo con un 0.9983043 de probabilidad.


¿cuánto aumenta la población mundial cada año?
1/1 [==============================] - 0s 44ms/step
Clasificación: NumericoCantidad con un 0.99858457 de probabilidad.


¿cuál es el nombre de pila del juez borsellino?
1/1 [==============================] - 0s 40ms/step
Clasificación: EntidadLugarPais con un 1.17116415e-05 de probabilidad.


¿cuántos miembros de la escolta murieron en el atentado contra el juez falcone?
1/1 [==============================] - 0s 38ms/step
Clasificación: NumericoCantidad con un 0.6959096 de probabilidad.


¿qué cargo detenta ariel sharon?
1/1 [==============================] - 0s 44ms/step
Clasificación: Definicion con un 0.8144713 de probabilidad.


¿qué año comenzó la intifada?
1/1 [==============================] - 0s 43ms/step
Clasificación: Definicion con un 0.99112386 de probabili